In [10]:
import pandas as pd
import numpy as np
import os
from draftSimulator import DraftSimulator
from fantasyTeam import Team
from seasonSimulator import SeasonSimulator
pd.options.mode.chained_assignment = None 

In [11]:
np.random.seed(42) 

script_dir = os.path.dirname(os.path.abspath(''))
data_dir = os.path.join(script_dir, 'data')
data_file_path = os.path.join(data_dir, 'ppr-adp-2023-updated.csv')
weekly_stats_path = os.path.join(data_dir, 'weekly-stats-2022.csv')

myTeam = Team("MahomiBear", 3)
leagueMembers = [
    ('Gridiron Gladiators', 10),
    ('Pigskin Pirates', 2),
    ('Touchdown Titans', 4),
    ('Blitzkrieg Bandits', 1),
    ('End Zone Avengers', 7),
    ('Hail Mary Heroes', 5),
    ('Fumble Force', 9),
    ('Red Zone Renegades', 6),
    ('Sack Masters', 8)
]

draft = DraftSimulator(path=data_file_path, myTeam=myTeam,
                            leagueMembers=leagueMembers, leagueSize=len(leagueMembers) + 1,
                            numRounds=16, stats=weekly_stats_path)

In [12]:
draftTeams = draft.teams.copy()
for _ in range(draft.numRounds):
    for team in draftTeams:
        # add conditional for 'me' when model created
        response = draft.otherTeamSelection(team)
        if response:
            player_name, playerTeam, position, byeWeek, status, avgadp = response
            team.addPickToRoster(position, player_name, draft.currentPick,
                                    avgadp, playerTeam, byeWeek, 0, status)
        else:
            print('error occured in selecting draft pick')
        draft.currentPick += 1
    draft.currentRound += 1
    draftTeams.reverse()
draft_picks = draft.draftPicksBoard

In [13]:
draft_picks

,team,draftRound,draftPick,player,position,playerTeam
0,Blitzkrieg Bandits,1,1,Christian McCaffrey,RB,SF
1,Pigskin Pirates,1,2,Austin Ekeler,RB,LAC
2,MahomiBear,1,3,Justin Jefferson,WR,MIN
3,Touchdown Titans,1,4,Ja'Marr Chase,WR,CIN
4,Hail Mary Heroes,1,5,Bijan Robinson,RB,ATL
...,...,...,...,...,...,...
155,Hail Mary Heroes,16,156,Kansas City Chiefs,DST,KC
156,Touchdown Titans,16,157,Dawson Knox,TE,BUF
157,MahomiBear,16,158,Brandon McManus,K,JAX
158,Pigskin Pirates,16,159,Gerald Everett,TE,LAC


In [14]:
waiver_wire_df = draft.constructWaiverWire()
waiver_wire_df.drop(columns=['Rank'], inplace=True)

In [15]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '..', 'data')
weekly_info_path = os.path.join(data_dir, 'simulator-weekly-info-2023.csv')
season = SeasonSimulator(teams=draft.teams, weekly_info_path=weekly_info_path, waiver_wire_df=waiver_wire_df)

In [16]:
season.simulate_season()

Team(Gridiron Gladiators)
K streaming: True
DST streaming: True
Waiver wire status: 0
   FantasyPosition             Name Position  PickNumber  \
0               QB     Dak Prescott       QB          71   
1              RB1    Miles Sanders       RB          50   
2              RB2   Khalil Herbert       RB          90   
3              WR1      CeeDee Lamb       WR          10   
4              WR2         AJ Brown       WR          11   
5               TE   Dallas Goedert       TE          70   
6             FLEX    Calvin Ridley       WR          30   
7                K  Harrison Butker        K         131   
8              DST    New York Jets      DST         130   
9              BE1     Deebo Samuel       WR          31   
10             BE2   Terry McLaurin       WR          51   
11             BE3   Brian Robinson       RB          91   
12             BE4    De'Von Achane       RB         110   
13             BE5  Elijah Mitchell       RB         111   
14            

In [17]:
ww = season.waiverWire.waiver_wire

In [18]:
season.team_dict['Fumble Force'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Justin Fields,QB,49,46.0,CHI,13,19.786667,ACT,18.20,9.64
1,RB1,Alvin Kamara,RB,69,69.0,NO,11,10.466667,ACT,15.82,16.00
2,RB2,Joe Mixon,RB,32,32.0,CIN,7,14.266667,ACT,13.50,15.10
3,WR1,Stefon Diggs,WR,9,10.0,BUF,13,7.933333,ACT,16.20,8.80
4,WR2,Nico Collins,WR,0,143.0,HOU,7,9.833333,ACT,15.65,0.00
5,TE,Dalton Kincaid,TE,0,135.0,BUF,13,4.800000,ACT,9.22,0.00
6,FLEX,DK Metcalf,WR,29,31.0,SEA,5,14.666667,ACT,14.00,12.80
7,K,Ka'imi Fairbairn,K,0,413.0,HOU,7,11.666667,ACT,8.34,6.00
8,DST,Buffalo Bills,DST,112,128.0,BUF,13,10.333333,ACT,8.40,9.00
9,BE1,Atlanta Falcons,DST,0,301.0,ATL,11,6.000000,ACT,7.80,7.00


In [19]:
season.team_dict['End Zone Avengers'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Dak Prescott,QB,0,83.0,DAL,7,14.993333,ACT,21.23,18.62
1,RB1,Tony Pollard,RB,14,15.0,DAL,7,6.300000,ACT,14.73,5.30
2,RB2,Najee Harris,RB,27,27.0,PIT,6,13.100000,ACT,10.16,13.80
3,WR1,Cooper Kupp,WR,7,7.0,LAR,10,16.233333,ACT,16.24,10.90
4,WR2,George Pickens,WR,0,79.0,PIT,6,21.100000,ACT,10.75,7.50
5,TE,Evan Engram,TE,74,85.0,JAX,9,12.100000,ACT,11.82,17.50
6,FLEX,Drake London,WR,0,57.0,ATL,11,6.966667,ACT,10.62,6.90
7,K,Justin Tucker,K,107,126.0,BAL,13,11.333333,ACT,8.44,15.00
8,DST,San Francisco 49ers,DST,94,100.0,SF,9,8.000000,ACT,9.00,3.00
9,BE1,Keenan Allen,WR,34,37.0,LAC,5,0.000000,INA,0.00,0.00


In [20]:
season.team_dict['Sack Masters'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Jake Browning,QB,0,800.0,CIN,7,17.813333,ACT,14.56,18.96
1,RB1,Jahmyr Gibbs,RB,33,34.0,DET,9,18.033333,ACT,15.41,24.80
2,RB2,Saquon Barkley,RB,8,9.0,NYG,13,11.300000,ACT,14.88,5.70
3,WR1,Davante Adams,WR,13,14.0,LV,13,21.033333,ACT,15.09,24.10
4,WR2,Jayden Reed,WR,0,228.0,GB,6,14.700000,ACT,11.68,17.20
5,TE,George Kittle,TE,53,53.0,SF,9,10.966667,ACT,12.83,7.40
6,FLEX,Travis Etienne,RB,28,30.0,JAX,9,13.933333,ACT,14.69,9.90
7,K,Jake Elliott,K,0,185.0,PHI,10,9.000000,ACT,9.14,5.00
8,DST,Cleveland Browns,DST,0,233.0,CLE,5,11.000000,ACT,8.40,13.00
9,BE1,James Cook,RB,0,75.0,BUF,13,15.500000,ACT,14.38,36.10


In [21]:
season.team_dict['Gridiron Gladiators'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,CJ Stroud,QB,0,225.0,HOU,7,4.306667,ACT,17.39,0.00
1,RB1,Ezekiel Elliott,RB,0,127.0,NE,11,14.033333,ACT,14.54,21.00
2,RB2,Chase Brown,RB,0,256.0,CIN,7,5.766667,ACT,0.00,0.00
3,WR1,CeeDee Lamb,WR,10,11.0,DAL,7,28.666667,ACT,22.37,25.20
4,WR2,AJ Brown,WR,11,13.0,PHI,10,11.300000,ACT,18.01,14.00
5,TE,Dallas Goedert,TE,70,71.0,PHI,10,12.300000,ACT,10.55,14.10
6,FLEX,Deebo Samuel,WR,31,36.0,SF,9,16.500000,ACT,16.44,9.40
7,K,Cairo Santos,K,0,451.0,CHI,13,9.000000,ACT,8.04,0.00
8,DST,Philadelphia Eagles,DST,0,118.0,PHI,10,4.000000,ACT,7.90,4.00
9,BE1,Calvin Ridley,WR,30,33.0,JAX,9,14.200000,ACT,12.47,25.80


In [22]:
season.team_dict['Blitzkrieg Bandits'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Josh Allen,QB,20,21.0,BUF,13,20.766667,ACT,20.96,21.16
1,RB1,Christian McCaffrey,RB,1,2.0,SF,9,26.633333,ACT,23.46,13.10
2,RB2,Isiah Pacheco,RB,80,77.0,KC,10,13.366667,ACT,14.23,29.50
3,WR1,Mike Evans,WR,81,78.0,TB,5,17.766667,ACT,15.51,10.00
4,WR2,DeAndre Hopkins,WR,40,49.0,TEN,7,7.433333,ACT,13.31,14.20
5,TE,Travis Kelce,TE,41,6.0,KC,10,7.266667,ACT,15.86,4.60
6,FLEX,Chris Godwin,WR,60,61.0,TB,5,18.833333,ACT,13.08,17.10
7,K,Jason Sanders,K,0,220.0,MIA,10,13.666667,ACT,8.02,7.00
8,DST,Chicago Bears,DST,0,275.0,CHI,13,10.333333,ACT,7.70,11.00
9,BE1,Jaylen Waddle,WR,21,22.0,MIA,10,11.466667,INA,0.00,0.00


In [23]:
season.team_dict['Hail Mary Heroes'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Kyler Murray,QB,0,188.0,ARI,14,19.140000,ACT,16.06,23.68
1,RB1,Bijan Robinson,RB,5,8.0,ATL,11,10.400000,ACT,14.42,11.60
2,RB2,James Conner,RB,0,67.0,ARI,14,22.133333,ACT,13.09,26.30
3,WR1,Garrett Wilson,WR,16,17.0,NYJ,7,10.533333,ACT,13.65,9.90
4,WR2,Tee Higgins,WR,25,29.0,CIN,7,16.666667,ACT,12.31,2.90
5,TE,Trey McBride,TE,0,274.0,ARI,14,13.366667,ACT,13.49,10.80
6,FLEX,David Njoku,TE,96,98.0,CLE,5,20.066667,ACT,12.68,17.40
7,K,Tyler Bass,K,0,155.0,BUF,13,7.333333,ACT,8.42,9.00
8,DST,Los Angeles Rams,DST,0,269.0,LAR,10,6.000000,ACT,8.00,8.00
9,BE1,Alexander Mattison,RB,56,56.0,MIN,13,1.633333,ACT,0.00,5.00


In [24]:
season.team_dict['MahomiBear'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Lamar Jackson,QB,43,35.0,BAL,13,25.820000,ACT,20.66,36.34
1,RB1,Zamir White,RB,0,198.0,LV,13,15.866667,ACT,13.32,15.60
2,RB2,Kenneth Walker,RB,38,42.0,SEA,5,14.433333,ACT,12.91,16.50
3,WR1,Justin Jefferson,WR,3,1.0,MIN,13,17.466667,ACT,18.62,10.90
4,WR2,DeVonta Smith,WR,23,25.0,PHI,10,11.300000,ACT,14.19,6.00
5,TE,Sam LaPorta,TE,0,163.0,DET,9,16.266667,ACT,12.03,15.40
6,FLEX,David Montgomery,RB,0,80.0,DET,9,12.533333,ACT,12.31,12.50
7,K,Jake Moody,K,0,215.0,SF,9,7.666667,ACT,9.12,9.00
8,DST,Tampa Bay Buccaneers,DST,0,295.0,TB,5,7.333333,ACT,7.30,1.00
9,BE1,DJ Chark,WR,0,181.0,CAR,7,11.133333,ACT,0.00,2.80


In [25]:
season.team_dict['Touchdown Titans'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Jalen Hurts,QB,24,24.0,PHI,10,21.513333,ACT,23.20,21.92
1,RB1,Jonathan Taylor,RB,44,43.0,IND,11,9.233333,ACT,14.98,0.00
2,RB2,Breece Hall,RB,37,40.0,NYJ,7,24.500000,ACT,14.59,2.80
3,WR1,Puka Nacua,WR,0,327.0,LAR,10,20.666667,ACT,15.98,10.30
4,WR2,Ja'Marr Chase,WR,4,3.0,CIN,7,5.833333,ACT,14.73,10.40
5,TE,Dalton Schultz,TE,0,119.0,HOU,7,9.266667,ACT,9.78,9.80
6,FLEX,Derrick Henry,RB,17,18.0,TEN,7,10.193333,ACT,13.00,5.00
7,K,Brandon Aubrey,K,0,356.0,DAL,7,7.333333,ACT,9.34,4.00
8,DST,New England Patriots,DST,117,139.0,NE,11,10.333333,ACT,4.80,7.00
9,BE1,Josh Downs,WR,0,258.0,IND,11,7.366667,ACT,8.79,4.40


In [26]:
season.standings

,Team,Wins,Losses,Points For,Points Against
0,Blitzkrieg Bandits,9,4,1716.58,1504.30
1,Pigskin Pirates,9,4,1499.68,1499.96
7,Sack Masters,8,5,1623.00,1525.34
6,End Zone Avengers,7,6,1522.64,1447.74
5,Red Zone Renegades,6,7,1721.22,1563.94
9,Gridiron Gladiators,6,7,1583.42,1614.16
3,Touchdown Titans,6,7,1566.08,1674.22
8,Fumble Force,6,7,1535.88,1510.86
2,MahomiBear,5,8,1507.66,1601.04
4,Hail Mary Heroes,3,10,1323.62,1658.22


In [27]:
waiver_wire_df = season.waiverWire.waiver_wire

In [28]:
for team in season.teams:
    print(team)
    print(team.waiverWireActivity)

Team(Blitzkrieg Bandits)
0.07140686396434298
Team(Pigskin Pirates)
0.15130099179959774
Team(MahomiBear)
0.3765990134708945
Team(Touchdown Titans)
0.08149943402927855
Team(Hail Mary Heroes)
0.11439286471096033
Team(Red Zone Renegades)
0.33806385394057653
Team(End Zone Avengers)
0.234066863105128
Team(Sack Masters)
0.26332029718706046
Team(Fumble Force)
0.2039643190957966
Team(Gridiron Gladiators)
0.060306051558721575


In [29]:
season.playoff_standings

,Team,Rank
0,Blitzkrieg Bandits,1
1,Pigskin Pirates,2
9,Gridiron Gladiators,3
6,End Zone Avengers,4
5,Red Zone Renegades,5
7,Sack Masters,6
3,Touchdown Titans,7
8,Fumble Force,8
4,Hail Mary Heroes,9
2,MahomiBear,10
